<a href="https://colab.research.google.com/github/masud407/Customer-Churn-Prediction-Using-Artificial-Neural-Network/blob/main/Titanic_dataset_prediction_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Import the required module to handle file uploads in Google Colab
from google.colab import files

# Open a file upload dialog in Colab, allowing users to upload files
uploaded = files.upload()

# The 'uploaded' variable is a dictionary where:
# - The keys are the file names
# - The values are the corresponding file contents (binary format)

Saving train.csv to train.csv


In [ ]:
import pandas as pd
df_train = pd.read_csv("train.csv")  # The file will be in the current Colab directory
df_train.head()  # Display the first few rows

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:

df_test = pd.read_csv("test.csv")  # The file will be in the current Colab directory
df_test.head()  # Display the first few rows

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:

# Concatenates df_train and df_test along rows (axis=0) without sorting column names
df = pd.concat([df_train, df_test], sort=False)

# The 'sort=False' ensures that column names retain their original order
# and are not sorted alphabetically when merging.

# Displays the first few rows of the concatenated DataFrame
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.isna().sum() #sum of NA is each column

,0
PassengerId,0
Survived,418
Pclass,0
Name,0
Sex,0
Age,263
SibSp,0
Parch,0
Ticket,0
Fare,1


In [ ]:
df = df.drop(['Name', 'Ticket'], axis = 1)

In [ ]:
   # Fill missing values (NaN) in the 'Age' column with the mean age of the dataset
df['Age'] = df['Age'].fillna(df['Age'].mean())

# Explanation:
# - df['Age'].mean() computes the average (mean) age from the existing values in the 'Age' column.
# - fillna() replaces all missing values (NaN) in 'Age' with this computed mean.
# - This helps maintain consistency in the dataset and prevents issues caused by missing values.


In [ ]:
# Fill missing values in the 'Cabin' column with 'X000' as a placeholder
df['Cabin'] = df['Cabin'].fillna('X000')
# - This ensures that all entries in 'Cabin' are non-null.
# - 'X000' acts as a placeholder, distinguishing missing values from real cabin numbers.
# - Useful for categorical processing or one-hot encoding later.

# Fill missing values in the 'Embarked' column with 'X' as a placeholder
df['Embarked'] = df['Embarked'].fillna('X')
# - 'X' represents an unknown embarkation point.
# - Prevents issues when working with categorical data (e.g., one-hot encoding).
# - This approach avoids dropping rows due to missing embarkation information.

# Fill missing values in the 'Fare' column with the mean fare
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
# - Missing fare values are replaced with the average fare.
# - Ensures that numerical calculations (e.g., scaling, modeling) remain valid.
# - Mean imputation is useful when the distribution is relatively normal.

In [ ]:
# Extract alphabetic characters (cabin letter) from the 'Cabin' column
df['cabin_letter'] = df['Cabin'].str.extract(r'([a-zA-Z]+)', expand=False)
# - Uses a regular expression (regex) to find one or more letters (A-Z, a-z).
# - This captures the **cabin section/letter** (e.g., 'C' from 'C123').
# - If no letters exist, the value will be NaN.

# Extract numeric characters (cabin number) from the 'Cabin' column
df['cabin_number'] = df['Cabin'].str.extract(r'(\d+)', expand=False)
# - Uses regex to find one or more digits (0-9).
# - This captures the **cabin number** (e.g., '123' from 'C123').
# - If no numbers exist, the value will be NaN.

# Drop the original 'Cabin' column since its information has been split
df = df.drop('Cabin', axis=1)
# - Removes the 'Cabin' column to avoid redundancy.
# - Keeps only the extracted `cabin_letter` and `cabin_number` for cleaner analysis.

In [ ]:
# Convert the 'cabin_letter' column into multiple binary columns (one-hot encoding)
# Each unique cabin letter (e.g., 'A', 'B', 'C') becomes a separate column (e.g., 'cabin_A', 'cabin_B', 'cabin_C')
df = pd.get_dummies(df, columns=['cabin_letter'], prefix='cabin')

# Convert the 'Embarked' column into multiple binary columns (one-hot encoding)
# Each embarkation point (e.g., 'C', 'Q', 'S') becomes a separate column (e.g., 'Embarked_C', 'Embarked_Q', 'Embarked_S')
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')

# Convert the 'Sex' column into multiple binary columns (one-hot encoding)
# The resulting columns will be 'Sex_female' and 'Sex_male', with 1 indicating the presence of that category
df = pd.get_dummies(df, columns=['Sex'], prefix='Sex')

df = df.drop('cabin_X', axis = 1)
df = df.drop('Embarked_X', axis = 1)

df['cabin_number'] = df['cabin_number'].fillna(0)
df['cabin_number'] = pd.to_numeric(df['cabin_number'])

#This feature gives an approximation of the average fare per class
df['Pclass_bin_Fare'] = df['Fare'] // df['Pclass']
#This feature might highlight whether being female in a lower or higher class correlates with survival rates
df['Pclass_bin_sex'] = df['Pclass'] - df['Sex_female']



In [ ]:
df_train = df[:len(df_train)]
df_test = df[len(df_train):]
df_test = df_test.drop('Survived', axis = 1)

In [ ]:
df_train.corr()['Survived']

,Survived
PassengerId,-0.005007
Survived,1.000000
Pclass,-0.338481
Age,-0.070323
SibSp,-0.035322
Parch,0.081629
Fare,0.257307
cabin_number,0.229756
cabin_A,0.022287
cabin_B,0.175095


In [ ]:
X = df_train.drop('Survived', axis = 1)
y = df_train['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)
y_train = np.reshape(y_train, (-1, 1))

In [ ]:
X_train.shape, y_train.shape

((712, 22), (712, 1))

In [ ]:
X_test.isna().sum()

,0
PassengerId,0
Pclass,0
Age,0
SibSp,0
Parch,0
Fare,0
cabin_number,0
cabin_A,0
cabin_B,0
cabin_C,0


In [ ]:
# Create an instance of the Logistic Regression model
model_1 = LogisticRegression()

# Train (fit) the Logistic Regression model using the training data (X_train and y_train)
# The model learns the relationship between input features (X_train) and target labels (y_train)
model_1.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = model_1.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.7932960893854749

In [ ]:
model_2 = XGBClassifier(enable_categorical = True)
model_2.fit(X_train, y_train)
y_pred = model_2.predict(X_test)

accuracy_score(y_test, y_pred)

0.770949720670391

In [ ]:
model_3 = RandomForestClassifier()
model_3.fit(X_train, y_train)
y_pred = model_3.predict(X_test)

accuracy_score(y_test, y_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8156424581005587

In [ ]:
model_4 = DecisionTreeClassifier()
model_4.fit(X_train, y_train)
y_pred = model_1.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.7932960893854749


In [ ]:
from sklearn.svm import SVC

model_5 = SVC()
model_5.fit(X_train, y_train)
y_pred = model_5.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.6759776536312849


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_6 = GradientBoostingClassifier()
model_6.fit(X_train, y_train)
y_pred = model_6.predict(X_test)

print(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7877094972067039


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_7 = KNeighborsClassifier(n_neighbors=5)
model_7.fit(X_train, y_train)
y_pred = model_7.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.6703910614525139


/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
# Use the trained model (model_3) to make predictions on the test dataset (df_test)
pred = model_3.predict(df_test)

# Create an empty DataFrame to store the final predictions
final = pd.DataFrame()

# Add the 'PassengerId' column from df_test to maintain reference to original test data
final['PassengerId'] = df_test['PassengerId']

# Store the predicted survival values in the 'Survived' column
final['Survived'] = pred

# Save the final DataFrame as a CSV file named 'output.csv' without the index column
# This ensures the output format is clean and ready for submission or further analysis
final.to_csv('output.csv', index=False)


In [ ]:
import os
print(os.getcwd())  # Should print: /content


/content


In [ ]:
from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
final.to_csv('/content/drive/My Drive/output.csv', index=False)
